# Treino do modelo

In [3]:
import os
import sys

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from Functions.Pipeline import pipeline_completa_skus
from Functions.FNC_Pro import lista_produtos,configurar_credenciais_bq


configurar_credenciais_bq(r'G:Drives compartilhados/Bases BI/epoca-230913-b478a9a0dd4c.json')

produtos = lista_produtos('G:/Drives compartilhados/Planilha de Impostos/BI/Planilha com Impostos.xlsm',Classificacao='A', Ativo='Sim',SKUS=['7172','10036','21120'])

# Executar pipeline completa
resultados, previsoes = pipeline_completa_skus(
    df_produtos=produtos,
    caminho_planilha_precos='C:/Users/joao.pcarvalho/Desktop/Git Repositórios/Forecast-vendas-e-elasticidade/Forecast/Precos_para_previsoes.xlsx',
    n_splits=10
)

print("Resultados consolidados:")
print(resultados.head())

print("Previsões consolidadas:")
print(previsoes.head())

Encontrados 3 SKUs com os filtros: Classificação='A', Ativo='Sim', SKUs='['7172', '10036', '21120']'
--- INICIANDO PIPELINE COMPLETA PARA TODOS OS SKUs ---
Total de SKUs para processar: 3
------------------------------------------------------------

>>> Processando SKU 7172 (105/3) <<<
SKU 7172: Histórico de dados ajustado para começar em 2022-09-29, o primeiro dia com vendas.
  [1/4] Treinando modelo de Validação Cruzada...
=== MODELO DE VALIDAÇÃO CRUZADA - SKU 7172 ===

Verificando qualidade dos dados...
 Executando validação cruzada temporal...
 Fold 1:
   Período teste: 2023-01-14 a 2023-04-22
   RMSE: 0.6172, WAPE: 14.47%
 Fold 2:
   Período teste: 2023-04-23 a 2023-07-30
   RMSE: 0.5658, WAPE: 11.83%
 Fold 3:
   Período teste: 2023-07-31 a 2023-11-06
   RMSE: 0.5102, WAPE: 10.71%
 Fold 4:
   Período teste: 2023-11-07 a 2024-02-13
   RMSE: 0.6253, WAPE: 13.18%
 Fold 5:
   Período teste: 2024-02-14 a 2024-05-22
   RMSE: 0.6122, WAPE: 12.87%
 Fold 6:
   Período teste: 2024-05-23 a 2

# Teste do modelo

In [4]:
from Functions.FNC_Previsoes import prever_demanda_com_modelos_salvos
import os
import pandas as pd

# --- Definir os caminhos para os modelos e a planilha de preços ---
caminho_pasta_modelos = os.path.join(project_root, 'Modelos')
caminho_planilha_precos = os.path.join(project_root, 'Forecast', 'Precos_para_previsoes.xlsx')

# --- Verificar se a pasta de modelos existe ---
if not os.path.isdir(caminho_pasta_modelos):
    print(f"ERRO: A pasta de modelos não foi encontrada em '{caminho_pasta_modelos}'")
    print("Por favor, execute o pipeline de treinamento primeiro para gerar os arquivos de modelo.")
    df_previsoes_salvas = pd.DataFrame()
else:
    # --- Gerar as previsões a partir dos modelos salvos ---
    df_previsoes_salvas = prever_demanda_com_modelos_salvos(
        caminho_pasta_modelos,
        caminho_planilha_precos
    )

    print("\nPrevisões geradas a partir dos modelos salvos:")
    display(df_previsoes_salvas.head(100))

--- INICIANDO PREVISÃO A PARTIR DE MODELOS SALVOS ---
Encontrados 3 SKUs no arquivo de preços.
  Processando SKU: 7172
  Processando SKU: 10036
  Processando SKU: 21120

--- PREVISÃO CONCLUÍDA ---

Previsões geradas a partir dos modelos salvos:


,Data,SKU,Preco,previsao_TSCV,previsao_SARIMAX
0,2025-09-18,7172,379,8.486701,4.225980
1,2025-09-19,7172,375,8.982769,6.637818
2,2025-09-20,7172,370,9.652108,6.479988
3,2025-09-21,7172,369,9.792986,7.476016
4,2025-09-22,7172,329,18.101402,18.424615
5,2025-09-23,7172,372,11.002123,7.178298
6,2025-09-24,7172,369,11.283827,9.302688
7,2025-09-25,7172,366,10.230502,9.959757
8,2025-09-26,7172,358,11.515581,12.234568
9,2025-09-27,7172,360,11.177167,9.001844
